In [1]:
import pandas as pd
import httplib2, requests
from bs4 import BeautifulSoup

# Load PMIDs

In [2]:
df = pd.read_csv("metadata.csv", encoding="latin-1")
df.head(3)

,PMID,DOI,KEY,SOURCE,AUTHORS,YEAR,MONTH,JOURNAL,TITLE,PAGES,...,NUM_COORDINATES,MNI_COORDINATES,BRAINMAP_ID,BEHAVIORAL_DOMAIN,EXPERIMENT,DESCRIPTION,IMPACT_FACTOR_2018,CITATIONS,CITATIONS_PER_YEAR,N_SUBJECTS
0,1402966,NaN,"Dolan R J, 1992",BrainMap,Dolan R J|Bench C J|Brown R G|Scott L C|Fristo...,1992,Sep,JOURNAL OF NEUROLOGY NEUROSURGERY & PSYCHIATRY,Regional cerebral blood flow abnormalities in ...,768-773,...,7.0,"-7.66,51.87,-8.33;-5.51,56.46,-4.28;-5.48,58.9...",6030020.0,"['Action.Rest', 'Action.Rest']","['Unimpaired > Impaired', 'Impaired > Unimpair...",Patients with depression who were cognitively ...,8.327,20,0.740741,10.0
1,1410086,NaN,"Bench C J, 1992",BrainMap,Bench C J|Friston K J|Brown R G|Scott L C|Frac...,1992,Aug,PSYCHOLOGICAL MEDICINE,The anatomy of melancholia - focal abnormaliti...,607-615,...,10.0,"-13.68,29.54,33.76;-15.76,23.53,38.75;-39.5,38...",6030018.0,"['Action.Rest', 'Action.Rest', 'Action.Rest']","['Normals > Depressed', 'Unimpaired > Impaired...",Patients with depression and normal subjects u...,5.641,91,3.370370,10.0
2,1448149,10.1038/360339a0,"Zatorre R J, 1992",BrainMap,Zatorre R J|Jones-Gotman M|Evans A C|Meyer E,1992,Nov,NATURE,Functional localization and lateralization of ...,339-340,...,6.0,"-21.56,5.58,-27.71;22.3,6.36,-21.58;18.98,41.4...",8110311.0,['Perception.Olfaction'],['Odor vs. Odorless'],Subjects underwent 2 conditions in which they ...,43.070,73,2.703704,11.0


In [3]:
pmids = list(df["PMID"].astype(int))
len(pmids)

18155

# Retrieve citation counts

In [4]:
pmid2citations = {}
http = httplib2.Http(".cache", disable_ssl_certificate_validation = True)
for i, pmid in enumerate(pmids):
    url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi?db=pubmed&id={}".format(int(pmid))
    try:
        response = BeautifulSoup(requests.get(url).text)
        citations = int(response.find(attrs={"name": "PmcRefCount"}).text.strip())
        pmid2citations[pmid] = citations
    except:
        print("\tUnable to retrieve citation count for {}".format(pmid))
        pmid2citations[pmid] = ""
    if i % 500 == 0:
        print("Processed {} articles".format(i))

Processed 0 articles
Processed 500 articles
Processed 1000 articles
Processed 1500 articles
Processed 2000 articles
Processed 2500 articles
Processed 3000 articles
Processed 3500 articles
Processed 4000 articles
Processed 4500 articles
Processed 5000 articles
Processed 5500 articles
Processed 6000 articles
Processed 6500 articles
Processed 7000 articles
Processed 7500 articles
Processed 8000 articles
Processed 8500 articles
Processed 9000 articles
Processed 9500 articles
Processed 10000 articles
Processed 10500 articles
Processed 11000 articles
Processed 11500 articles
Processed 12000 articles
Processed 12500 articles
Processed 13000 articles
Processed 13500 articles
Processed 14000 articles
Processed 14500 articles
Processed 15000 articles
Processed 15500 articles
Processed 16000 articles
Processed 16500 articles
Processed 17000 articles
Processed 17500 articles
Processed 18000 articles


In [5]:
citations = [pmid2citations[pmid] for pmid in pmids]
citations_per_year = citations / (2020 - df["YEAR"])

# Save to file

In [6]:
date = 200711

In [7]:
output = pd.DataFrame({"PMID": pmids, 
                       "CITATIONS": citations, 
                       "YEAR": df["YEAR"],
                       "CITATIONS_PER_YEAR": citations_per_year})
output.to_csv("citations_{}.csv".format(date), index=None)

In [8]:
df["CITATIONS"] = citations
df["CITATIONS_PER_YEAR"] = citations_per_year

In [9]:
df.to_csv("metadata.csv", index=None)